In [1]:
import numpy as np
import theano
import theano.tensor as tt
THEANO_FLAGS='optimizer=fast_compile'



Important Equations

$$n = \mbox{Number of States in the differential equation.  The dimension of } y$$

$$m = \mbox{Number of parameters in the differential equation.  The dimension of } p$$

$$y' = f(y,t,p)$$

$$ \dfrac{d}{dt} \left( \dfrac{\partial y}{\partial p} \right) = \dfrac{\partial f}{\partial y} \dfrac{\partial y}{\partial p} + \dfrac{\partial f}{\partial p} $$


$$ \dfrac{\partial f}{\partial y} = \begin{bmatrix} 
\dfrac{\partial f_1}{\partial y_1} & \dots & \dfrac{\partial f_1}{ \partial y_n} \\
\vdots & \ddots & \vdots \\
\dfrac{\partial f_n}{\partial y_1} & \dots & \dfrac{\partial f_n}{\partial y_n}
\end{bmatrix} \in \mathbb{R}^{n \times n}$$

$$ \dfrac{\partial y}{\partial p} = \begin{bmatrix} 
\dfrac{\partial y_1}{\partial p_1} & \dots & \dfrac{\partial y_1}{ \partial p_m} \\
\vdots & \ddots & \vdots \\
\dfrac{\partial y_n}{\partial p_1} & \dots & \dfrac{\partial y_n}{\partial p_m}
\end{bmatrix} \in \mathbb{R}^{n \times m}$$

$$ \dfrac{\partial f}{\partial y} = \begin{bmatrix} 
\dfrac{\partial f_1}{\partial p_1} & \dots & \dfrac{\partial f_1}{ \partial p_m} \\
\vdots & \ddots & \vdots \\
\dfrac{\partial f_n}{\partial p_1} & \dots & \dfrac{\partial f_n}{\partial p_m}
\end{bmatrix} \in \mathbb{R}^{n \times m}$$


In [2]:

def augment_system(ode_func):


    '''Function to create augmented system.

    Take a function which specifies a set of differential equations and return
    a compiled function which allows for computation of gradients of the
    differential equation's solition with repsect to the parameters.

    Args:
        ode_func (function): Differential equation.  Returns array-like

    Returns:
        system (function): Augemted system of differential equations.

    '''

    #Present state of the system
    t_y = tt.dvector('y')

    #Parameter(s).  Should be vector to allow for generaliztion to multiparameter
    #systems of ODEs
    t_p = tt.dvector('p')

    #Time.  Allow for non-automonous systems of ODEs to be analyzed
    t_t = tt.dscalar('t')

    #Present state of the gradients:
    #Will always be 0 unless the parameter is the inital condition
    #Entry i,j is partial of y[i] wrt to p[j]
    dydp = tt.dmatrix('dydp')

    #Stack the results of the ode_func
    #TODO: Does this behave the same of ODE is scalar?
    f_tensor = tt.stack(ode_func(t_y, t_t, t_p))

    #Now compute gradients
    J = tt.jacobian(f_tensor,t_y)

    Jdfdy = tt.dot(J, dydp)

    grad_f = tt.jacobian(f_tensor, t_p)

    #This is the time derivative of dydp
    ddt_dydp = Jdfdy + grad_f

    #TODO: Ensure system is reshaped on the odeint call?
    system = theano.function(
            inputs=[t_y, t_t, t_p, dydp],
            outputs=[f_tensor, ddt_dydp],on_unused_input='ignore')

    return system


In [26]:

def augment_system(ode_func):


    '''Function to create augmented system.

    Take a function which specifies a set of differential equations and return
    a compiled function which allows for computation of gradients of the
    differential equation's solition with repsect to the parameters.

    Args:
        ode_func (function): Differential equation.  Returns array-like

    Returns:
        system (function): Augemted system of differential equations.

    '''
    
    t_n = tt.scalar('n', dtype = 'int32')
    t_m = tt.scalar('m', dtype = 'int32')

    #Present state of the system
    t_y = tt.dvector('y')

    #Parameter(s).  Should be vector to allow for generaliztion to multiparameter
    #systems of ODEs
    t_p = tt.dvector('p')

    #Time.  Allow for non-automonous systems of ODEs to be analyzed
    t_t = tt.dscalar('t')

    #Present state of the gradients:
    #Will always be 0 unless the parameter is the inital condition
    #Entry i,j is partial of y[i] wrt to p[j]
    dydp_vec = tt.vector('dydp')   
    
    dydp = dydp_vec.reshape((t_n,t_m))


    #Stack the results of the ode_func
    #TODO: Does this behave the same of ODE is scalar?
    f_tensor = tt.stack(ode_func(t_y, t_t, t_p))

    #Now compute gradients
    J = tt.jacobian(f_tensor,t_y)

    Jdfdy = tt.dot(J, dydp)

    grad_f = tt.jacobian(f_tensor, t_p)

    #This is the time derivative of dydp
    ddt_dydp = (Jdfdy + grad_f).flatten()

    #TODO: Ensure system is reshaped on the odeint call?
    system = theano.function(
            inputs=[t_y, t_t, t_p, dydp_vec ,t_n, t_m],
            outputs=[f_tensor, ddt_dydp],on_unused_input='ignore')

    return system


In [28]:
def test_ode_func_1(y,t,p):

    #Scalar ODE, one parameter
    #Inspired by first order pharmacokineitc models
    #y' = exp(-t)-p*y
    return np.exp(-t) - p[0]*y[0]


sys_1 = augment_system(test_ode_func_1)

n = 1
m = 1

sys_1(np.zeros(shape = n),0, np.array([0]), np.zeros(shape = n*m),n,m)

/Users/demetri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: stack(*tensors) interface is deprecated, use stack(tensors, axis=0) instead.
  if __name__ == '__main__':


[array([1.]), array([0.])]

In [29]:
def test_ode_func_2(y,t,p):

    #Scalar ODE, two parameters
    #Inspired by first order pharmacokineitc models
    #y' = D/V*k_a*exp(-k_a*t)-k*y

    return p[0]*np.exp(-p[0]*t)-p[1]*y[0]

sys_2 = augment_system(test_ode_func_2)

n = 1
m = 2

y = np.zeros(n)
t = 0
p = np.random.exponential(size = m)
dydp = np.zeros(shape = n*m)

sys_2(y,t,p,dydp,n,m)


/Users/demetri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: stack(*tensors) interface is deprecated, use stack(tensors, axis=0) instead.
  if __name__ == '__main__':


[array([5.06506623]), array([1., 0.])]

In [30]:

def test_ode_func_3(y,t,p):

    #Vector ODE, scalar parameter
    #non-dimensionalized SIR model
    #S' = -R_0*S*I, I' = R_0*S*I - I


    ds = -p[0]*y[0]*y[1]
    di = p[0]*y[0]*y[1] - y[1]

    return [ds,di]

sys_3 = augment_system(test_ode_func_3)


n = 2
m = 1

y = np.zeros(n)
t = 0
p = np.random.exponential(size = m)
dydp = np.zeros(n*m)

sys_3(y,t,p,dydp,n,m)



[array([-0.,  0.]), array([0., 0.])]

In [31]:
def test_ode_func_4(y,t,p):

    #Vector ODE, vector paramter
    #Inspired by SIR model
    #S' = -beta*S*I, I' = beta*S*I - gamma*I


    ds = -p[0]*y[0]*y[1]
    di = p[0]*y[0]*y[1] - p[1]*y[1]

    return [ds,di]


sys_4 = augment_system(test_ode_func_4)


n = 2
m = 2

y = np.zeros(n)
t = 0
p = np.random.exponential(size = n)
dydp = np.zeros(n*m)

sys_4(y,t,p,dydp,n,m)

[array([-0.,  0.]), array([0., 0., 0., 0.])]

array([[1]])